In [1]:
import heapq
import numpy as np
import folium
import simplekml
import pandas as pd
import random

In [4]:
# Define the soldier's start and destination GPS "Example"
start_lat, start_lon = 24.7136, 46.6753  # Soldier's starting position
goal_lat, goal_lon = 24.7146, 46.6783  # Soldier's target destination

# Generate Random Landmine Locations
num_landmines = 15  # Change the number of landmines for testing
landmine_positions = [
    (
        round(random.uniform(start_lat, goal_lat), 6),
        round(random.uniform(start_lon, goal_lon), 6)
    ) for _ in range(num_landmines)
]

# Save new landmine locations to CSV for reproducibility
df_landmines = pd.DataFrame(landmine_positions, columns=["Latitude", "Longitude"])
df_landmines.to_csv("landmine_locations.csv", index=False)

print("✅ New landmine locations generated and saved to 'landmine_locations.csv'")

# Define search grid parameters
grid_size = (150, 150)  
cell_size_m = 2  
meters_per_degree = 111320  

# Convert GPS to grid coordinates
def gps_to_grid(lat, lon):
    x = int((lat - start_lat) * meters_per_degree / cell_size_m)
    y = int((lon - start_lon) * meters_per_degree / (cell_size_m * np.cos(np.radians(start_lat))))
    
    # Ensure X and Y are within grid bounds
    x = max(0, min(x, grid_size[0] - 1))
    y = max(0, min(y, grid_size[1] - 1))
    return x, y

# Convert grid to GPS coordinates
def grid_to_gps(x, y):
    lat = start_lat + (x * cell_size_m / meters_per_degree)
    lon = start_lon + (y * cell_size_m / (meters_per_degree * np.cos(np.radians(start_lat))))
    return lat, lon

# Convert landmine GPS coordinates to grid coordinates
landmine_grid_positions = [gps_to_grid(lat, lon) for lat, lon in landmine_positions]

# Create the grid map (0 = safe, 1 = landmine)
grid_map = np.zeros(grid_size)

# Reduce Landmine Buffer for More Flexibility
landmine_buffer = 2  
for x, y in landmine_grid_positions:
    for dx in range(-landmine_buffer, landmine_buffer + 1):
        for dy in range(-landmine_buffer, landmine_buffer + 1):
            if 0 <= x+dx < grid_size[0] and 0 <= y+dy < grid_size[1]:
                grid_map[x+dx, y+dy] = 1  # Mark area around landmine as unsafe

# Convert start & goal to grid
start_grid = gps_to_grid(start_lat, start_lon)
goal_grid = gps_to_grid(goal_lat, goal_lon)

# Ensure goal stays within grid bounds
goal_grid = (
    max(0, min(goal_grid[0], grid_size[0] - 1)),
    max(0, min(goal_grid[1], grid_size[1] - 1))
)

# Ensure Goal is Not Inside a Landmine Zone
if 0 <= goal_grid[0] < grid_size[0] and 0 <= goal_grid[1] < grid_size[1]:
    if grid_map[goal_grid[0], goal_grid[1]] == 1:
        print("❌ The goal location is inside a landmine area. Adjusting goal position...")
        
        # Find the nearest safe cell within a 3-cell radius
        found_safe_goal = False
        for i in range(-3, 4):
            for j in range(-3, 4):
                new_x, new_y = goal_grid[0] + i, goal_grid[1] + j
                if (0 <= new_x < grid_size[0] and 
                    0 <= new_y < grid_size[1] and 
                    grid_map[new_x, new_y] == 0):  # Ensure it's a safe position
                    goal_grid = (new_x, new_y)
                    found_safe_goal = True
                    print(f"✅ New goal position set to {goal_grid}")
                    break
            if found_safe_goal:
                break  # Exit loop once a safe goal is found

print("✅ Final goal position is within bounds and safe.")

# A* Algorithm for Safe Path Planning
class AStarSafePath:
    def __init__(self, grid, start, goal):
        self.grid = grid
        self.start = start
        self.goal = goal
        self.open_list = []
        self.closed_list = set()
        self.came_from = {}
        self.g_score = {node: float('inf') for node in np.ndindex(grid.shape)}
        self.g_score[start] = 0
        self.f_score = {node: float('inf') for node in np.ndindex(grid.shape)}
        self.f_score[start] = self.heuristic(start, goal)
        heapq.heappush(self.open_list, (self.f_score[start], start))

    def heuristic(self, node, goal):
        return abs(node[0] - goal[0]) + abs(node[1] - goal[1])  # Manhattan distance

    def reconstruct_path(self):
        path = []
        current = self.goal
        while current in self.came_from:
            path.append(current)
            current = self.came_from[current]
        path.append(self.start)
        path.reverse()
        return path

    def find_safe_path(self):
        # 8-directional movement
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
        while self.open_list:
            _, current = heapq.heappop(self.open_list)

            if current == self.goal:
                return self.reconstruct_path()

            self.closed_list.add(current)
            for dx, dy in directions:
                neighbor = (current[0] + dx, current[1] + dy)
                if (0 <= neighbor[0] < self.grid.shape[0] and 
                    0 <= neighbor[1] < self.grid.shape[1] and 
                    self.grid[neighbor] == 0 and 
                    neighbor not in self.closed_list):
                    
                    tentative_g_score = self.g_score[current] + 1
                    if tentative_g_score < self.g_score[neighbor]:
                        self.came_from[neighbor] = current
                        self.g_score[neighbor] = tentative_g_score
                        self.f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, self.goal)
                        heapq.heappush(self.open_list, (self.f_score[neighbor], neighbor))

        return None  # No safe path found

# Find the safe path
astar_safe_path = AStarSafePath(grid_map, start_grid, goal_grid)
safe_path_grid = astar_safe_path.find_safe_path()

if safe_path_grid is None:
    print("❌ No valid path found! The destination might be unreachable due to landmines.")
    exit()  # Stop execution to prevent further errors

# Convert safe path to GPS coordinates
safe_path_gps = [grid_to_gps(x, y) for x, y in safe_path_grid]

print("✅ Safe path generated successfully!")

# Save the safe path to CSV
df_path = pd.DataFrame(safe_path_gps, columns=["Latitude", "Longitude"])
df_path.to_csv("safe_path.csv", index=False)

print("✅ Safe path saved as 'safe_path.csv'.")

# Export Safe Path to KML for Google Earth
kml = simplekml.Kml()
linestring = kml.newlinestring(name="Safe Path for Soldiers")
linestring.coords = safe_path_gps  # Add the full path
linestring.style.linestyle.color = simplekml.Color.green
linestring.style.linestyle.width = 3
kml.save("safe_path.kml")

print("✅ KML file 'safe_path.kml' created! Open it in Google Earth.")

# Visualize the Safe Path on an Interactive Map
map_folium = folium.Map(location=[start_lat, start_lon], zoom_start=18)

# Draw the full safe path
if safe_path_gps:
    folium.PolyLine(safe_path_gps, color="blue", weight=5, tooltip="Safe Path").add_to(map_folium)
else:
    print("❌ No safe path GPS coordinates to visualize!")

# Add the goal marker
folium.Marker([goal_lat, goal_lon], tooltip="Safe Goal", icon=folium.Icon(color="blue")).add_to(map_folium)

# Add landmine locations (red markers)
for lat, lon in landmine_positions:
    folium.Marker(location=[lat, lon], icon=folium.Icon(color="red", icon="exclamation-triangle")).add_to(map_folium)

# Save the interactive map as an HTML file
map_folium.save("optimized_safe_path.html")

print("✅ Interactive safe path map saved as 'optimized_safe_path.html'. Open it in a browser.")


✅ New landmine locations generated and saved to 'landmine_locations.csv'
✅ Final goal position is within bounds and safe.
✅ Safe path generated successfully!
✅ Safe path saved as 'safe_path.csv'.
✅ KML file 'safe_path.kml' created! Open it in Google Earth.
✅ Interactive safe path map saved as 'optimized_safe_path.html'. Open it in a browser.
